# ENEXIS Graduation Project

#### Create a model predicting the 'opwek' based on demographical data

### Collect Initial Data

In [12]:
import pandas as pd
import sys
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
# from sklearn import datasets, linear_model
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import train_test_split

### Get the data

In [13]:
combined_dataset_name = 'combined_CBS_dataset_84583.csv'
df = pd.read_csv(filepath_or_buffer = combined_dataset_name,
                         sep                = ';',
                         decimal            = ',',
                         thousands          = '.',
                         encoding           = 'unicode_escape')                                   # 1

## Models
### Linear model Y=opgesteld vermogen, X=Aantal huishoudens

In [14]:
train_set, test_set = train_test_split(df, test_size=0.2)
train_set_for_plots = train_set
test_set_for_plots = test_set
print(f"training set: {len(train_set)}\n    test set: {len(test_set)}")

y_column = 'Opgesteld_vermogen'
columns_to_keep = ['HuishoudensTotaal_28']
# HuishoudensTotaal_28
colomns_to_drop = [
    'Opgesteld_vermogen', 
    'Peildatum',
    'CBS_Buurtcode',
    'Aantal_aansluitingen_in_CBS-buurt',
    'Aantal_aansluitingen_met_opwekinstallatie',
    'Netbeheerder',
    'Provincie',
    'CBS_Buurt', 
    'Gemeente', 
    'WijkenEnBuurten',
    'Gemeentenaam_1',
    'SoortRegio_2',
    'MeestVoorkomendePostcode_113'
    ]

Y_train_set = train_set[y_column]
X_train_set = train_set.drop(colomns_to_drop, axis=1).copy()
# X_train_set = train_set[columns_to_keep]
Y_test_set = train_set[y_column]
X_test_set = train_set.drop(colomns_to_drop, axis=1).copy()
# X_test_set = train_set[columns_to_keep]


training set: 2218
    test set: 555


In [15]:
X_test_set.head()

,HuishoudensTotaal_28,GemiddeldeHuishoudensgrootte_32,Bevolkingsdichtheid_33,GemiddeldeWOZWaardeVanWoningen_35,PercentageEengezinswoning_36,Koopwoningen_40,InBezitWoningcorporatie_42,InBezitOverigeVerhuurders_43,BouwjaarVanaf2000_46,Gemiddeld_opleidingsniveau,Bedrijfsvestigingen_per_huishuidens,PersonenautoSPerHuishouden_102,AfstandTotSchool_108,Omgevingsadressendichtheid_116
1018,600,2.2,1484.0,198.0,82.0,56.0,38.0,6.0,17.0,1.923810,0.141667,1.1,0.5,1114.0
914,60,3.0,73.0,418.0,99.0,88.0,0.0,12.0,31.0,2.066667,0.750000,1.0,1.5,280.0
2079,1760,2.3,1048.0,242.0,86.0,54.0,41.0,5.0,8.0,1.839228,0.164773,1.2,0.5,786.0
1733,75,2.7,32.0,539.0,100.0,90.0,0.0,10.0,15.0,1.937500,0.666667,1.0,1.5,95.0
1359,5285,2.3,1949.0,267.0,85.0,71.0,23.0,5.0,18.0,1.914823,0.208136,1.3,0.7,1143.0


In [16]:
lin_reg = LinearRegression()
lin_reg.fit(X_train_set, Y_train_set)

LinearRegression()

In [17]:
import altair as alt

alt.renderers.enable('mimetype')

base = alt.Chart(train_set_for_plots).mark_circle().encode(
    alt.X('HuishoudensTotaal_28',
     title='# huishoudens'),
       alt.Y('Opgesteld_vermogen',
     title='Opgesteld vermogen')
)

linear_fit = [
    base.transform_regression(
        "HuishoudensTotaal_28", "Opgesteld_vermogen", method="linear"
    )
    .mark_line()   
]

alt.layer(base, *linear_fit)

<VegaLite 4 object>

If you see this message, it means the renderer has not been properly enabled
for the frontend that you are using. For more information, see
https://altair-viz.github.io/user_guide/troubleshooting.html


#### Check the quality of the model

In [18]:
# Make predictions using the testing set
y_pred = lin_reg.predict(X_test_set)

# The coefficients
print("Coefficients: \n", lin_reg.coef_)
# The mean squared error
print("Mean squared error: %.2f" % mean_squared_error(Y_test_set, y_pred))
# The coefficient of determination: 1 is perfect prediction
print("Coefficient of determination: %.2f" % r2_score(Y_test_set, y_pred))


Coefficients: 
 [ 5.71097332e-01  6.72694707e+01 -5.15843891e-02 -2.30699434e-01
  3.31755769e+00  2.68491277e+00  8.76311515e-01 -2.09504570e+00
  1.65487246e+00  1.19650152e+02  3.44060739e+01  2.07421710e+01
 -6.36064701e+01 -8.90857675e-02]
Mean squared error: 413518.65
Coefficient of determination: 0.27
